# Demo 02 - Gaps

In this demo, we will look at gaps in data.  Looking at gaps in data is important because it shows us where we might be missing individual data points.  If this is a set which should be complete, such as check numbers or invoice numbers, it might give you a telling indicator that there might be an issue.

For this demo, we will use pyodbc and ipython-sql.  pyodbc is an ODBC driver for Python, whereas ipython-sql allows you to use "sql magic" in Jupyter.  You can just as easily run the queries in SQL Server Management Studio if you prefer.

First, let's use pip to install pyodbc and ipython-sql and prep them for load.

In [ ]:
!pip install pyodbc

To load pyodbc, we can use the **import** statement.

In [ ]:
import pyodbc

In [ ]:
!pip install ipython-sql

To use SQL magic, we will need to run the following load command.

In [ ]:
%reload_ext sql

From here on out, I can use the *%sql* command to run a single-line SQL command.  I can also use the *%%sql* command to run multi-line SQL commands.

The first thing I want to connect to the OutlierDetection database.  I have already created an ODBC connection pointing to localhost.OutlierDetection.  You do not need to use a pre-defined ODBC connection, but when connecting to SQL Server, I've found it easier to use a pre-defined connection.

In [ ]:
%sql mssql+pyodbc://ForensicAccounting

In our data, invoice number is the key value we use for tracking money.  This number should never have any gaps in it.

To test this, we will run a query which looks for gaps in the data.  Note that this uses syntax which Microsoft introduced in SQL Server 2012.  If you are using SQL Server 2008 R2 or earlier, you would need to use a different set of syntax to solve the gaps and islands problem.

In [ ]:
%%sql WITH C AS
(
	SELECT
		li.LineItemID AS CurrentLineItemID,
		LEAD(li.LineItemID) OVER (ORDER BY li.LineItemID) AS NextLineItemID
	FROM dbo.LineItem li
)
SELECT
	CurrentLineItemID + 1 AS rangestart,
	NextLineItemID- 1 AS rangeend
FROM C
WHERE
	NextLineItemID - CurrentLineItemID > 1;


As an alternative to gap analysis, you can join to a tally (or numbers) table and look for cases in which there is no matching result in the invoices table.

This query first builds up a tally table, which is extremely fast for values up to around 10,000 records.  After about 10,000 tables, it might make more sense to build a tally table and store it on disk.

In [ ]:
%%sql WITH
	t1 AS (SELECT 1 N UNION ALL SELECT 1 N),
	t2 AS (SELECT 1 N FROM t1 x, t1 y),
	t3 AS (SELECT 1 N FROM t2 x, t2 y),
	t4 AS (SELECT 1 N FROM t3 x, t3 y),
	Tally AS (SELECT ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) N FROM t4 x, t4 y)
SELECT
	t.N
FROM Tally t
	LEFT OUTER JOIN dbo.LineItem li
		ON t.N = li.LineItemID
WHERE
	li.LineItemID IS NULL
	AND t.N >= (SELECT MIN(LineItemID) FROM dbo.LineItem imin)
	AND t.N <= (SELECT MAX(LineItemID) FROM dbo.LineItem imax)
ORDER BY
	T.N;